# Bibtex to DF and tidy

**One off code for data rationalization and normalization.**

* df
* author list
* publisher list
* journal list
* files
* normalization

item 241 has weird annot


## Provenance
* Created new 2025-05-16 at Bignor!

In [657]:
from pathlib import Path 
import pandas as pd
import re
from functools import partial
import pprint as pp

import Levenshtein

import scratch as sc
import great2_bibtex as scbt

from greater_tables import GT
deff = lambda x: str(x)
fGT = partial(GT, large_ok=True, formatters={'index': lambda x: str(x), 'year': lambda x: f'{int(x):d}'})
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 0 ns (started: 2025-05-20 08:09:51 +01:00)


## Manual read

In [106]:
p = Path('books.bib')
p = Path('library.bib')
p.exists()

True

time: 0 ns (started: 2025-05-18 17:44:45 +01:00)


In [562]:
b = sc.Bib2df(p, False)
len(b.df)

6118

time: 578 ms (started: 2025-05-19 21:15:42 +01:00)


In [ ]:
b.df.head(2).T

In [563]:
b.contents()

,column,nonna,distinct
0,type,6118,10
1,tag,6118,6076
2,abstract,2518,2376
3,author,6113,4856
4,doi,2687,2538
5,file,5951,5917
6,issn,2425,826
7,journal,3475,1224
8,keywords,2999,2052
9,pages,3885,2748


time: 172 ms (started: 2025-05-19 21:15:44 +01:00)


In [782]:
for f in b.contents().column: #  ['address', 'primaryClass', 'editor', 'series', 'chapter', 'school', 'organization', 'howpublished']: 
    display(fGT(b.df[f].value_counts().sort_values()[::-1].head(30), caption=f))
    print()

type,count
article,"4,980"
book,418
techreport,413
misc,133
unpublished,59
incollection,59
inproceedings,44
phdthesis,10
Report,1
manual,1


tag,count
Danielsson2013,3
Cummins1988,3
Kokonendji2006,3
Cheridito2009b,2
Deck2016,2
Bailey1967,2
Kupper2009a,2
LUMBERMENSMUTUALCASUALTYCOMPANY2006,2
Landsman1998a,2
Boneh2003a,2


abstract,count
,"3,600"
500-1000 citations,22
1000-2500 citations,15
250-500 citations,10
5K-10K Citations,8
"JSTOR is a not-for-profit service that helps scholars, researchers, and students discover, use, and build upon a wide range of content in a trusted digital archive. We use information technology and tools to increase productivity and facilitate new forms of scholarship. For more information about JSTOR, please contact support@jstor.org.",6
2500-5000 citations,5
"Predicting the binding mode of flexible polypeptides to proteins is an important task that falls outside the domain of applicability of most small molecule and protein−protein docking tools. Here, we test the small molecule flexible ligand docking program Glide on a set of 19 non-\(\alpha\)-helical peptides and systematically improve pose prediction accuracy by enhancing Glide sampling for flexible polypeptides. In addition, scoring of the poses was improved by post-processing with physics-based implicit solvent MM- GBSA calculations. Using the best RMSD among the top 10 scoring poses as a metric, the success rate (RMSD ≤ 2.0 {\AA} for the interface backbone atoms) increased from 21% with default Glide SP settings to 58% with the enhanced peptide sampling and scoring protocol in the case of redocking to the native protein structure. This approaches the accuracy of the recently developed Rosetta FlexPepDock method (63% success for these 19 peptides) while being over 100 times faster. Cross-docking was performed for a subset of cases where an unbound receptor structure was available, and in that case, 40% of peptides were docked successfully. We analyze the results and find that the optimized polypeptide protocol is most accurate for extended peptides of limited size and number of formal charges, defining a domain of applicability for this approach.",4
The asymptotic behaviour of the residual life time at time t is investigated (for t rightarrow infty). We derive weak limit laws and their domains of attraction and treat rates of convergence and moment convergence. The presentation exploits the close similarity with extreme value theory.,3
"Precautionary saving has been linked to the property of prudence, and the property of temperance has been used to show how the presence of an unavoidable risk affects one's behaviour towards a second risk. These two higher order risk effects also play key roles in aversion to negative skewness and to kurtosis, respectively. This article presents a laboratory experiment to determine whether subjects are prudent and/or temperate. The experiment is based upon preferences over lottery pairs in simple 50–50 gambles. Subjects are asked in which of two states of nature they would prefer to receive a zero-mean gamble. For prudence, the choices are between a lower and higher wealth outcome. For temperance, the choices are between a state with no other risk and a state with a second (independent) risk. The results show behavioural evidence for prudence, but they also show evidence of intemperate behaviour. Implications of these results for both expected-utility and non-expected-utility models are examined. [ABSTRACT FROM AUTHOR]",3


author,count
{Aon Benfield},90
"Mildenhall, Stephen J",34
{A. M. Best},32
Aon plc,23
{LUMBERMENS MUTUAL CASUALTY COMPANY},21
"Delbaen, Freddy",17
"Major, John A.",13
"Wang, Shaun S.",13
"Badger, Stephen G",12
"Bauer, Daniel and Zanjani, George",11


doi,count
,"3,431"
10.1017/CBO9781107415324.004,21
10.4324/9780203358061,7
10.1007/s13398-014-0173-7.2,5
10.2307/252115,4
10.1111/j.1467-9965.2009.00364.x,3
10.1111/j.1467-937X.2010.00605.x,3
10.1007/s00780-013-0225-4,3
10.2307/3315588,3
10.1016/j.jeconom.2012.08.011,3


...

pages,count
,"2,233"
1--12,30
1--15,30
1--28,27
1--21,27
1--11,25
1--25,24
1--30,24
1--26,24
1--14,24


title,count
{Reinsurance Market Outlook},15
{Global Insurance Market Opportunities},5
{Insurance-Linked Securities},5
"{Fat tails, VaR and subadditivity}",4
{Catastrophe Risk Tolerance Study},4
{The Aon Benfield Aggregate},3
{The Lindsay transform of natural exponential families},3
{Exploring higher order risk effects},3
{Annual Global Climate and Catastrophe Report},3
{Stochastic Loss Reserving Using Bayesian MCMC Models},3


volume,count
,"3,143"
1,81
3,61
2,52
7,50
21,49
6,47
8,46
4,45
12,45


ValueError: invalid literal for int() with base 10: ''

time: 829 ms (started: 2025-05-20 22:58:41 +01:00)


In [ ]:
# run fillna = False
df = b.contents(True)

In [452]:
b.df.file.value_counts().sort_values().tail(10)

file
:C\:/S/Library/Paouris/2012_Small ball probability estimates for log-concave measures.pdf:pdf                                                                                                                      2
:C\:/S/Library/Kokonendji, Seshadri/1994_The Lindsay transform of natural exponential families.pdf:pdf;:C\:/S/Library/Kokonendji, Seshadri/2006_The Lindsay transform of natural exponential families.pdf:pdf      2
:C\:/S/Library/Aon Benfield/2010_Aon Benfield Aggregate, H1 2010.pdf:pdf                                                                                                                                           2
:C\:/S/Library/Gagnon, Sack/2014_Monetary policy with abundant liquidity.pdf:pdf                                                                                                                                   2
:C\:/S/Library/Crouhy, Galai, Mark/2006_The Essentials of Risk Management.pdf:pdf                                                              

time: 32 ms (started: 2025-05-19 18:43:38 +01:00)


In [376]:
base_cols = ['tag', 'type', 'author', 'title', 'year']

time: 0 ns (started: 2025-05-18 22:28:48 +01:00)


In [377]:
b.df.loc[b.df.file.isna(), base_cols]

,tag,type,author,title,year
38,keynesgeneral,book,"Keynes, John Maynard","{General theory of employment, interest and mo...",1936
53,Mildenhall2018c,article,"Mildenhall, Stephen J",{Every law invariant coherent risk measure has...,2018
75,Eling2017,article,"Eling, Martin and Jia, Ruo",{Recent Research Developments Affecting Nonlif...,2017
114,daykin94,book,"Daykin, C. D. and Pentikainen, T. and Pesonen, M.",{Practical Risk Theory for Actuaries},1994
133,Crouhy2010,article,"Crouhy, Michel and Galai, Dan and Mark, Robert",{Risk Exposures},2010
...,...,...,...,...,...
5865,hull01,book,"Hiriart-Urruty, Jean-Baptiste and Lemar{\'{e}}...",{Fundamentals of Convex Analysis},2001
5931,eeckhoudtgs04,book,"Eeckhoudt, Louis and Gollier, Christian and Sc...",{Economic and Financial Decisions under Risk},2005
6006,Strain1997,book,"Strain, Robert W.",{Reinsurance},1997
6039,jacob02,book,"Jacob, Niels",{Pseduo Differential Operators & Markov Proces...,2002


time: 16 ms (started: 2025-05-18 22:28:54 +01:00)


In [381]:
b = sc.Bib2df(p, True)

time: 312 ms (started: 2025-05-18 22:31:10 +01:00)


In [ ]:
fGT(b.df.loc[b.df['mendeley-tags'].str.contains('Book', case=False), base_cols + ['mendeley-tags', 'file']])

6118

time: 15 ms (started: 2025-05-18 21:18:34 +01:00)


## File demographics

### Searching

In [274]:
b.df.loc[b.df.author.str.contains('Bloch|Deligne', regex=True, case=False).fillna(False)]

,type,tag,abstract,author,doi,file,issn,journal,keywords,pages,...,month,annote,primaryClass,address,editor,series,chapter,school,organization,howpublished
754,article,Alarcon2016,It is nearly impossible to separate two interl...,"Alarc{\'{o}}n, H{\'{e}}ctor and Salez, Thomas ...",10.1103/PhysRevLett.116.015502,:C\:/S/Library/Alarc{\'{o}}n et al/2016_Self-A...,10797114,Physical Review Letters,,1--5,...,,,,,,,,,,
1381,article,Bloch2014,,"Bloch, Spencer",,:C\:/S/Library/Bloch/2014_Algebraic Cycles.pdf...,,,,,...,,,,,,,,,,
3086,article,Deligne2012,,"Deligne, Pierre and Sullivan, Pierre",,":C\:/S/Library/Deligne, Sullivan/2012_Division...",,,,,...,,,,,,,,,,
3837,article,Delignette-Muller2015,The package fitdistrplus provides functions fo...,"Delignette-Muller, Marie Laure and Dutang, Chr...",10.18637/jss.v064.i04,":C\:/S/Library/Delignette-Muller, Dutang/2015_...",1548-7660,Journal of Statistical Software,"bootstrap,censored data,distributions,maximum ...",,...,,,,,,,,,,
5406,article,Bloch1984,,"Bloch, Spencer",10.1515/crll.1984.350.94,:C\:/S/Library/Bloch/1984_Algebraic Cycles and...,14355345,Journal fur die Reine und Angewandte Mathematik,,94--108,...,,,,,,,,,,


time: 47 ms (started: 2025-05-18 21:29:46 +01:00)


### Duplicated tag

In [780]:
fGT(b.df.loc[b.df.tag.duplicated(keep=False), ['tag', 'author', 'title']].sort_values('tag'), tabs=[5, 15, 25, 40])

index,tag,author,title
2288,Aliprantis2006,"Aliprantis, Charalambos D and Border, Kim",{Infinite Dimensional Analysis: A Hitchhiker's Guide}
742,Aliprantis2006,"Aliprantis, Charalambos D and Burkinshaw, Owen",{Positive Operators}
418,AonBenfield2016,"Mildenhall, Stephen J",{Insurance Risk Study: The First Ten Years}
5541,AonBenfield2016,{Aon Benfield},{Cyber Opportunity and Future Growth}
1793,Avanzi2011,"Avanzi, Benjamin",{Optimal dividends and capital injections in the dual model with diffusions}
3225,Avanzi2011,"Avanzi, Benjamin and Cassar, Luke C and Wong, Bernard",{Modelling Dependence in Insurance Claims Processes with L{\'{e}}vy Copulas Modelling Dependence in Insurance Claims Processes with L{\'{e}}vy Copulas}
2938,Babbel1997,"Babbel, David F",{Review of Two Paradigms for the Market Value of Liabilities by Reitano}
2023,Babbel1997,"Babbel, David F.","{Discussion of ``Two Paradigms for The Market Value of Liabilities, Robert R. Reitano''}"
3857,Bailey1967,"Bailey, Robert A",{Underwriting Profit From Investments}
4082,Bailey1967,"Bailey, Robert A",{Underwriting Profits From Investments}


time: 78 ms (started: 2025-05-20 22:51:43 +01:00)


### Distinct Journals

In [499]:
j = b.distinct('journal')
j[:10]

['116 Mich. L. Re.',
 '1Mathematical Intelligencer',
 '1Stochastic Processes and their Applications',
 '2003 IEEE Computer Society Conference on Computer Vision and Pattern Recognition, 2003. Proceedings.',
 '2012 International Conference on Systems and Informatics (ICSAI2012)',
 '2Variance',
 '30th Annual Symposium on Foundations of Computer Science',
 '31st International Congress of Actuaries',
 '34th International Conference on Machine Learning, ICML 2017',
 'A Modern Course on Statistical Distributions in Scientific Work']

time: 16 ms (started: 2025-05-19 20:33:54 +01:00)


In [506]:
jdf = b.journal_map_df()

Decode errors: len(self.last_decode) = 7
time: 906 ms (started: 2025-05-19 20:38:28 +01:00)


In [507]:
b.last_decode

['tex_to_unicode DECODE ERR | "Algorithms–ESA 2012      " | \'latin-1\' codec can\'t encode character \'\\u2013\' in position 10: ordinal not in range(256)',
 'tex_to_unicode DECODE ERR | "Journal of Economic Perspectives—Volume" | \'latin-1\' codec can\'t encode character \'\\u2014\' in position 32: ordinal not in range(256)',
 'tex_to_unicode DECODE ERR | "Lecture Notes, http://galton. uchicago. edu/∼ {\\ldots}" | \'latin-1\' codec can\'t encode character \'\\u223c\' in position 44: ordinal not in range(256)',
 'tex_to_unicode DECODE ERR | "Model Regulation Service—October" | \'latin-1\' codec can\'t encode character \'\\u2014\' in position 24: ordinal not in range(256)',
 'tex_to_unicode DECODE ERR | "Sankhy{\\"{A}}\x81: The Indian Journal of Statistics, Series A" | \'ascii\' codec can\'t decode byte 0x81 in position 13: ordinal not in range(128)',
 'tex_to_unicode DECODE ERR | "Sankhyā: The Indian Journal of Statistics" | \'latin-1\' codec can\'t encode character \'\\u0101\' in pos

time: 0 ns (started: 2025-05-19 20:38:29 +01:00)


In [509]:
fGT(jdf.head(100))

index,original,unicoded
0,116 Mich. L. Re.,116 Mich. L. Re.
1,1Mathematical Intelligencer,1Mathematical Intelligencer
2,1Stochastic Processes and their Applications,1Stochastic Processes and their Applications
3,"2003 IEEE Computer Society Conference on Computer Vision and Pattern Recognition, 2003. Proceedings.","2003 IEEE Computer Society Conference on Computer Vision and Pattern Recognition, 2003. Proceedings."
4,2012 International Conference on Systems and Informatics (ICSAI2012),2012 International Conference on Systems and Informatics (ICSAI2012)
5,2Variance,2Variance
6,30th Annual Symposium on Foundations of Computer Science,30th Annual Symposium on Foundations of Computer Science
7,31st International Congress of Actuaries,31st International Congress of Actuaries
8,"34th International Conference on Machine Learning, ICML 2017","34th International Conference on Machine Learning, ICML 2017"
9,A Modern Course on Statistical Distributions in Scientific Work,A Modern Course on Statistical Distributions in Scientific Work


time: 203 ms (started: 2025-05-19 20:39:26 +01:00)


In [454]:
fGT(b.df.journal.value_counts().head(100))

journal,count
,"2,643"
Insurance: Mathematics and Economics,145
Journal of Risk and Insurance,136
ASTIN Bulletin,108
SSRN Electronic Journal,60
North American Actuarial Journal,58
Mathematical Finance,50
Risk Management and Insurance Review,46
Journal of Finance,45
The Journal of Finance,44


time: 172 ms (started: 2025-05-19 18:45:14 +01:00)


### Distinct authors

In [495]:
b = sc.Bib2df(p)
adf = b.author_map_df()
len(b.df), len(adf), len(adf) / 60, b.last_decode[0]

Decode errors: len(self.last_decode) = 17


(6118,
 8160,
 136.0,
 "tex_to_unicode DECODE ERR: Caicedo, Andr´es Eduardo \t'ascii' codec can't decode byte 0xb4 in position 13: ordinal not in range(128)")

time: 3.86 s (started: 2025-05-19 20:09:56 +01:00)


In [496]:
fGT(adf.query('original != initials').head(100))

index,original,unicoded,initials,proposed
3,"A, Hidalgo C{\'{e}}sar","A, Hidalgo César","A, Hidalgo César","A, Hidalgo César"
5,"ACIA, Dublin","ACIA, Dublin","A. C. I.A, Dublin","A. C. I.A, Dublin"
13,"Aas, K","Aas, K","Aas, K.","Aas, K."
20,"Abbott, B P","Abbott, B P","Abbott, B. P.","Abbott, B. P."
27,"Abraham, R","Abraham, R","Abraham, R.","Abraham, R."
40,"Acharya, Viral V","Acharya, Viral V","Acharya, Viral V.","Acharya, Viral V."
41,"Ackground, I I B","Ackground, I I B","Ackground, I. I. B.","Ackground, I. I. B."
42,"Acosta, A","Acosta, A","Acosta, A.","Acosta, A."
52,"Adams, Michael B","Adams, Michael B","Adams, Michael B.","Adams, Michael B."
69,"Aharon, M","Aharon, M","Aharon, M.","Aharon, M."


time: 281 ms (started: 2025-05-19 20:10:00 +01:00)


In [498]:
adf.to_csv('author_mapper.csv', encoding='utf-8')

time: 63 ms (started: 2025-05-19 20:10:32 +01:00)


## Files

In [550]:
b._file_field_df = None
ffdf = b.file_field_df

time: 969 ms (started: 2025-05-19 21:03:13 +01:00)


In [551]:
len(ffdf)

6079

time: 0 ns (started: 2025-05-19 21:03:15 +01:00)


In [553]:
all(ffdf.index == ffdf.idx)

True

time: 0 ns (started: 2025-05-19 21:03:21 +01:00)


In [554]:
ffdf.head()

,idx,drive,file,type
i,,,,
1,1,C\,/S/Library/Schmidt/2003_The true invariant of ...,pdf
2,2,C\,/S/Library/Bahnemann/2015_Distributions for Ac...,pdf
2,2,C\,/S/Library/Bahnemann/2015_Distributions for Ac...,pdf
3,3,C\,/S/Library/Khoshnevisan/2011_Topics in Probabi...,pdf
4,4,C\,/S/Library/Mahul/2003_Efficient Risk Sharing w...,pdf


time: 32 ms (started: 2025-05-19 21:03:23 +01:00)


In [541]:
len(ffdf)

6079

time: 0 ns (started: 2025-05-19 21:00:54 +01:00)


In [560]:
ffdf.groupby(by='idx').file.count().to_frame('count').query('count > 1').sort_values('count')

,count
idx,
2,2
76,2
380,2
420,2
453,2
...,...
2797,3
3077,3
4823,3


time: 31 ms (started: 2025-05-19 21:04:20 +01:00)


In [561]:
_.sum()

count    246
dtype: int64

time: 0 ns (started: 2025-05-19 21:04:33 +01:00)


In [ ]:
missing = []
for i, r in ffdf.iterrows():
    if not Path(r.file).exists():
        print(i, r.file)
        missing.append([i, r.file])

In [526]:
missing[:3]

[[5,
  "/S/Library/Jiang, Koo, Wang/2025_The Road Not Taken Evaluating the Consequences of the U.S.'s Non-Adoption of IFRS in 2012.pdf"],
 [37,
  "/S/Library/Filipovi{\\'{c}}, Svindland/2008_Optimal capital and risk allocations for law- and cash-invariant convex functions.pdf"],
 [39,
  "/S/Library/Heras, Balb{\\'{a}}s, Vilar/2012_Conditional tail expectation and premium calculation.pdf"]]

time: 0 ns (started: 2025-05-19 20:44:06 +01:00)


In [525]:
len(missing)

479

time: 0 ns (started: 2025-05-19 20:43:58 +01:00)


In [314]:
ffdf.head().drop(columns='i').file.values

array(['/S/Library/Schmidt/2003_The true invariant of an arbitrage free portfolio.pdf',
       '/S/Library/Bahnemann/2015_Distributions for Actuaries.pdf',
       '/S/Library/Bahnemann/2015_Distributions for Actuaries(2).pdf',
       '/S/Library/Khoshnevisan/2011_Topics in Probability Levy Processes.pdf',
       '/S/Library/Mahul/2003_Efficient Risk Sharing within a Catastrophe Insurance Pool.pdf'],
      dtype=object)

time: 0 ns (started: 2025-05-18 21:50:10 +01:00)


In [533]:
actual_files = [str(i.as_posix()) for i in Path('\\S\\Library').rglob('*.pdf')]
len(actual_files)

6128

time: 2.14 s (started: 2025-05-19 20:54:30 +01:00)


In [534]:
actual_files[:3]

['/S/Library/451 Research/2014_Data Platforms Map.pdf',
 '/S/Library/451 Research/2015_Data Platform Map.pdf',
 "/S/Library/A. M. Best/2004_Best's Insolvency Study, PropertyCasualty U.S. Insurers 1969-2002.pdf"]

time: 16 ms (started: 2025-05-19 20:54:33 +01:00)


In [532]:
missing[:3]

[[5,
  "/S/Library/Jiang, Koo, Wang/2025_The Road Not Taken Evaluating the Consequences of the U.S.'s Non-Adoption of IFRS in 2012.pdf"],
 [37,
  "/S/Library/Filipovi{\\'{c}}, Svindland/2008_Optimal capital and risk allocations for law- and cash-invariant convex functions.pdf"],
 [39,
  "/S/Library/Heras, Balb{\\'{a}}s, Vilar/2012_Conditional tail expectation and premium calculation.pdf"]]

time: 0 ns (started: 2025-05-19 20:54:00 +01:00)


In [537]:
ans = []
for i, m in missing:
    best_match = min(actual_files,
        key=lambda alt: Levenshtein.distance(m, alt))
    ans.append([i, m, best_match])

time: 19.7 s (started: 2025-05-19 20:57:50 +01:00)


In [ ]:
ansdf = pd.DataFrame(ans, columns=['idx', 'missing', 'match'])

In [539]:
fGT(ansdf.head(100))

index,idx,missing,match
0,5,"/S/Library/Jiang, Koo, Wang/2025_The Road Not Taken Evaluating the Consequences of the U.S.'s Non-Adoption of IFRS in 2012.pdf","/S/Library/Jiang, Koo, Wang/2025_The Road Not Taken Evaluating the Consequences of the U.S.’s Non-Adoption of IFRS in 2012.pdf"
1,37,"/S/Library/Filipovi{\'{c}}, Svindland/2008_Optimal capital and risk allocations for law- and cash-invariant convex functions.pdf","/S/Library/Filipović, Svindland/2008_Optimal capital and risk allocations for law- and cash-invariant convex functions.pdf"
2,39,"/S/Library/Heras, Balb{\'{a}}s, Vilar/2012_Conditional tail expectation and premium calculation.pdf","/S/Library/Heras, Balbás, Vilar/2012_Conditional tail expectation and premium calculation.pdf"
3,43,"/S/Library/M{\""{u}}ller/1998_Comparing risks with unbounded distributions.pdf",/S/Library/Müller/1998_Comparing risks with unbounded distributions.pdf
4,81,"/S/Library/B{\""{u}}hlmann, Jewell/1978_Operational Risk.pdf","/S/Library/Bühlmann, Jewell/1978_Operational Risk.pdf"
5,112,"/S/Library/B{\""{a}}uerle, M{\""{u}}ller/2006_Stochastic orders and risk measures Consistency and bounds.pdf","/S/Library/Bäuerle, Müller/2006_Stochastic orders and risk measures Consistency and bounds.pdf"
6,115,"/S/Library/Garmin - Fenix, Garmin/2013_Owner ' s Manual.pdf","/S/Library/Garmin - Fenix, Garmin/2013_Owner ’ s Manual.pdf"
7,120,"/S/Library/Klartag, Vershynin/2007_Small ball probability and Dvoretzky's Theorem.pdf","/S/Library/Klartag, Vershynin/2007_Small ball probability and Dvoretzky’s Theorem.pdf"
8,125,"/S/Library/K{\""{u}}chler, S{\o}rensen/1994_Exponential families of stochastic processes and L{\'{e}}vy processes.pdf","/S/Library/Küchler, Sørensen/1994_Exponential families of stochastic processes and Lévy processes.pdf"
9,145,/S/Library/Rosi{\'{n}}ski/2011_Infinitely Divisible Processes.pdf,/S/Library/Rosiński/2011_Infinitely Divisible Processes.pdf


time: 234 ms (started: 2025-05-19 20:58:37 +01:00)


In [ ]:
# List of alternatives
alts = ["apple", "banana", "orange", "grape", "pineapple"]

# String to match
s = "applle"

# Find the best match using Levenshtein distance
best_match = min(alts, key=lambda alt: Levenshtein.distance(s, alt))

# Calculate the distance (edit distance)
distance = Levenshtein.distance(s, best_match)

print(f"Best match: {best_match}, Distance: {distance}")

Best match: apple, Distance: 1
time: 125 ms (started: 2025-05-19 20:51:25 +01:00)


# Google Scholar 

In [567]:
import webbrowser
import urllib.parse
import pyperclip

def open_google_scholar(s):
    query = urllib.parse.quote_plus(s)
    url = f'https://scholar.google.com/scholar?q={query}'
    webbrowser.open_new_tab(url)


time: 31 ms (started: 2025-05-20 07:21:54 +01:00)


In [569]:
open_google_scholar('Mildenhall major "pricing insurance risk"')

time: 328 ms (started: 2025-05-20 07:22:20 +01:00)


In [617]:
bibtex = pyperclip.paste()
d = sc.Bib2df.parse_gs(bibtex)
pp.pprint(d)

{'author': 'Mildenhall, Stephen J and Major, John A',
 'publisher': 'John Wiley \\& Sons',
 'tag': 'mildenhall2022pricing',
 'title': 'Pricing insurance risk: Theory and practice',
 'type': 'book',
 'year': '2022'}
time: 0 ns (started: 2025-05-20 07:54:57 +01:00)


In [640]:
def dict_to_bibtex(d):
    """Create Bibtex string from dictionary d."""
    standard_types = ['report', 'article', 'book', 'incollection',
 'inproceedings',
 'manual',
 'misc',
 'phdthesis',
 'techreport',
 'unpublished']
    min_fields = ['type', 'tag', 'title', 'year']
    fails = []
    for f in min_fields:
        if f not in d:
            fails.append(f'missing {f}')
    if fails:
        raise ValueError('Missing required fields\n' + '\n'.join(fails))
    d = d.copy()
    t = d.pop('type')
    tag = d.pop('tag')
    tag = tag[0].upper() + tag[1:]
    title = d.pop('title')
    assert t in standard_types, f'Type must be a standard type not {t}'
    s = f'@{t}{{{tag},\ntitle = {{{{{title}}}}},\n' + '\n'.join(f'{k} = {{{v}}},' for k, v in d.items()) + '\n}'
    return s

time: 15 ms (started: 2025-05-20 07:59:52 +01:00)


In [641]:
print(dict_to_bibtex(d))

@book{Mildenhall2022pricing,
title = {{Pricing insurance risk: Theory and practice}},
author = {Mildenhall, Stephen J and Major, John A},
year = {2022},
publisher = {John Wiley \& Sons},
}
time: 0 ns (started: 2025-05-20 07:59:52 +01:00)


In [633]:
print(b.txt[207:1600])

@article{Schmidt2003,
abstract = {It is shown that the arbitrage free portfolio paradigm being applied to a portfolio with an arbitrary number of shares N allows for the extended solution in which the option price F depends on N. However the resulting stock hedging expense Q = MF (where M is the number of options in the portfolio) does not depend on whether N is treated as an independent variable or as a parameter. Therefore the stock hedging expense is the true invariant of the arbitrage free portfolio paradigm. {\textcopyright} 2002 Elsevier Science B.V. All rights reserved.},
author = {Schmidt, Anatoly B.},
doi = {10.1016/S0378-4371(02)01551-0},
file = {:C\:/S/Library/Schmidt/2003_The true invariant of an arbitrage free portfolio.pdf:pdf},
issn = {03784371},
journal = {Physica A: Statistical Mechanics and its Applications},
keywords = {Arbitrage free portfolio,Option pricing},
pages = {535--538},
title = {{The true invariant of an arbitrage free portfolio}},
volume = {320},
year = {

In [645]:
b = sc.Bib2df(p, True)

time: 703 ms (started: 2025-05-20 08:02:47 +01:00)


In [658]:
fGT(b.df.loc[b.df.annote.str.contains('[Cc]itations', regex=True), ['author', 'title', 'year', 'annote']])

index,author,title,year,annote
3414,"Csoka, Peter and Pinter, Miklos",{On the Impossibility of Fair Risk Allocation},2016,"From Duplicate 2 (On the Impossibility of Fair Risk Allocation - Csoka, Peter; Pinter, Miklos) 27 Google Citations"
3996,"Modigliani, Franco and Miller, Merton H","{The cost of capital, corporate fiance and the theory of investment}",1958,22000 citations
5271,"Camerer, Colin F. and Ho, Teck Hua",{Violations of the betweenness axiom and nonlinearity in probability},1994,755 GSCitations
6028,"Lecun, Yann and Bengio, Yoshua and Hinton, Geoffrey",{Deep learning},2015,"2. ""Deep learning"" (2015) Nature 16,750 citations This paper stands out not just because of its high number of citations, but because there was a difference of more than 10,000 between its citation count and the second most-cited Nature paper in the 2019 Google Scholar Metrics report. Authored by 2018 Turing Award winners, Yann LeCun, Yoshua Bengio and Geoffrey Hinton – known collectively as the 'Godfathers of AI' – the paper is a seminal review of the potential of the AI technologies."


time: 62 ms (started: 2025-05-20 08:10:15 +01:00)


In [ ]:
fGT(b.df.loc[b.df.abstract.str.contains('[Cc]itations?', regex=True), ['author', 'title', 'year', 'abstract']])

In [757]:
# regex to extract group like 1000, 2K, 1,000, 1K-2K etc.
pat = r'([\dKk\- ]+)\+? +(?: Google | GS )?[Cc]itations?'

time: 0 ns (started: 2025-05-20 08:45:38 +01:00)


In [752]:
df = b.df.copy()

time: 0 ns (started: 2025-05-20 08:43:06 +01:00)


In [758]:
v = df.loc[:, 'abstract'].str.extract(pat)[0].str.strip().dropna()
v

124           195
146          6652
228     1000-2500
229          5345
233            66
          ...    
5990     500-1000
5996     500-1000
6035    1000-2500
6036    1000-2500
6108          108
Name: 0, Length: 197, dtype: object

time: 203 ms (started: 2025-05-20 08:45:40 +01:00)


In [759]:
df.loc[2780, 'abstract']

'500+ citations'

time: 0 ns (started: 2025-05-20 08:45:41 +01:00)


In [761]:
re.search(pat, df.loc[2780, 'abstract']).groups(1)

('500',)

time: 0 ns (started: 2025-05-20 08:45:51 +01:00)


In [762]:
def parse_citations(s):
    try:
        return int(s)
    except ValueError:
        pass
        # print('not int')

    try:
        if s.find('-') > 0:
            l, r = s.split('-')        
            l = int(l.strip())
            r = int(r.strip())
            return (l + r) // 2
    except ValueError:
        pass
        # print('not range')

    s = s.lower()
    try:
        if s.find('k') > 0:
            return parse_citations(s.replace('k', '000')) 
    except ValueError:
        # print('not K')
        pass

    return f'Unable to parse {s}'
    # return -99

time: 0 ns (started: 2025-05-20 08:45:53 +01:00)


In [686]:
fGT(df.loc[822, ['title', 'abstract']])

index,822
title,{Identity-based encryption from the Weil pairing}
abstract,7000 GS citations!


time: 47 ms (started: 2025-05-20 08:27:35 +01:00)


In [763]:
w = v.map(parse_citations)
w.head()

124     195
146    6652
228    1750
229    5345
233      66
Name: 0, dtype: object

time: 16 ms (started: 2025-05-20 08:46:01 +01:00)


In [764]:
v[2780]

'500'

time: 16 ms (started: 2025-05-20 08:46:03 +01:00)


In [765]:
df.loc[w[w.str.contains('Unab', regex=True, na=False)].index, ['author', 'year', 'abstract']]

,author,year,abstract
5431,"Bunz, Benedikt and Bootle, Jonathan and Boneh,...",2018,Boneh - 71444 citations...h = 108; Stanford


time: 32 ms (started: 2025-05-20 08:46:05 +01:00)


In [721]:
w[w.str.contains('195', regex=True, na=False)]

Series([], Name: 0, dtype: object)

time: 16 ms (started: 2025-05-20 08:35:23 +01:00)


In [711]:
v[124]

'195 '

time: 15 ms (started: 2025-05-20 08:33:16 +01:00)


In [767]:
b.df.to_csv('dm.csv', encoding='utf-8-sig')

time: 313 ms (started: 2025-05-20 22:42:10 +01:00)


In [768]:
test = pd.DataFrame([(i, chr(i)) for i in range(65, 10000)], columns=['ord', 'char'])

time: 15 ms (started: 2025-05-20 22:44:48 +01:00)


In [770]:
test.to_csv('dm.csv', encoding='utf-8-sig')

time: 16 ms (started: 2025-05-20 22:45:16 +01:00)


In [776]:
test2 = pd.read_csv('dm2.csv', encoding='utf-8-sig', index_col=0)

time: 16 ms (started: 2025-05-20 22:49:19 +01:00)


In [777]:
test2.head()

,ord,char
0,65,A
1,66,B
2,67,C
3,68,D
4,69,E


time: 0 ns (started: 2025-05-20 22:49:20 +01:00)


In [779]:
all(test2==test)

True

time: 0 ns (started: 2025-05-20 22:49:40 +01:00)


In [772]:
fGT(test.head())

index,ord,char
0,65,A
1,66,B
2,67,C
3,68,D
4,69,E


time: 15 ms (started: 2025-05-20 22:48:29 +01:00)


In [689]:
# apply extract and parse
fGT(v.map(parse_citations))

index,0
124,195
146,"6,652"
228,"1,750"
229,"5,345"
233,66
380,196
493,750
501,42
550,114
601,Unable to parse


time: 391 ms (started: 2025-05-20 08:28:21 +01:00)


In [ ]:
fGT(b.df[['tag', 'author', 'year', 'title']].tail(20))

In [612]:
d = {'a': 10, 'b': 20}
d

{'a': 10, 'b': 20}

time: 0 ns (started: 2025-05-20 07:51:22 +01:00)


In [613]:
d.pop('a')

10

time: 0 ns (started: 2025-05-20 07:51:28 +01:00)


In [614]:
d

{'b': 20}

time: 0 ns (started: 2025-05-20 07:51:29 +01:00)


## Bibtex parser read

In [42]:
scbt.LIBRARY_BIB, p, p.exists(), scbt.BOOKS_BIB

('library.bib', WindowsPath('library.bib'), True, 'books.bib')

time: 0 ns (started: 2025-05-17 10:12:17 +01:00)


In [43]:
bdb = scbt.bibtex_read() # too slow... scbt.BOOKS_BIB)

time: 437 ms (started: 2025-05-17 10:12:29 +01:00)


In [44]:
len(bdb)

TypeError: object of type 'BibDatabase' has no len()

time: 47 ms (started: 2025-05-17 10:12:36 +01:00)
